In [1]:
import sys
sys.path.append('/Users/mnannan/dev/supelec/ngsa/assignments/Assignement 2 ')

In [7]:
from features.colaboration import extract_source_target_authors_list, build_collaboration_df, merge_authors,CollaborationShortestPaths
from features.authors import ExtractAuthorsList

In [8]:
from core.io.loader import *

In [9]:
import igraph

In [10]:
import numpy as np

In [11]:
from sklearn.utils.extmath import cartesian


In [12]:
import pandas as pd

In [13]:
train = get_data_with_node_information('train', data_dir='../data')

In [14]:
train = ExtractAuthorsList().transform(train)

In [15]:
train = CollaborationShortestPaths().transform(train)

In [20]:
train.loc[411789][['source_authors', 'target_authors', 'collaboration_paths']].values

array(['Sung-Won Kim, Won T. Kim, Young-Jai Park, , Hyeonjoon Shin',
       'Won T. Kim, Julian Lee, , Young Jai Park',
       list([1.0, 2.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 2.0, 1.0])],
      dtype=object)

In [17]:
train.collaboration_paths

0                                                      None
510736    [5.0, 6.0, 5.0, 4.0, 5.0, 4.0, 4.0, 5.0, 4.0, ...
451434    [5.0, 4.0, 5.0, 6.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...
168455                       [7.0, 7.0, 6.0, 6.0, 6.0, 5.0]
170930    [4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 4.0, 5.0, 4.0, ...
21430     [4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, ...
426916                                      [5.0, 4.0, 5.0]
201084                       [6.0, 5.0, 6.0, 5.0, 4.0, 5.0]
167792                                                 None
192101        [5.0, 4.0, 4.0, 6.0, 5.0, 5.0, 6.0, 5.0, 5.0]
126225                       [5.0, 4.0, 5.0, 5.0, 4.0, 4.0]
514327                                      [8.0, 7.0, 7.0]
222625        [6.0, 6.0, 5.0, 7.0, 7.0, 6.0, 7.0, 7.0, 6.0]
486060                                                 None
484078                                                 None
411789    [1.0, 2.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...
28303                                   

In [314]:
def create_colaboration_graph(df):
    """6s"""
    source_authors_list , _ =  extract_source_target_authors_list(train)
    source_authors = pd.DataFrame(source_authors_list)
    source_authors = source_authors.rename({0:'paper_id', 1:'author'}, axis='columns')

    authors_mapping = {value:key for key, value in source_authors.author.drop_duplicates().reset_index(drop=True).items()}
    
    colaboration = build_colaboration_df(source_authors)
    colaboration['merge_authors'] = merge_authors(colaboration.author_1, colaboration.author_2)
    colaboration = colaboration.sort_values(['paper_id', 'author_1', 'author_2']).drop_duplicates(['paper_id', 'merge_authors'])
    colaboration_edges = []
    for author_1, author_2 in zip(colaboration.author_1, colaboration.author_2):
        colaboration_edges.append((authors_mapping[author_1], authors_mapping[author_2]))
    return igraph.Graph(colaboration_edges)

In [315]:
st = time.time()
graph_colaboration = create_colaboration_graph(train)
print(time.time() - st)

1.8907582759857178


In [316]:
graph_colaboration.vcount()

11456

In [317]:
graph_colaboration.ecount()

40353

In [318]:
def extract_colaboration_shortest_paths_matrix(graph):
    """33 s"""
    return np.matrix(graph_colaboration.shortest_paths())

In [319]:
st = time.time()
shortest_path = extract_colaboration_shortest_paths_matrix(graph_colaboration)
print(time.time() - st)

33.276445388793945


In [323]:
def extract_shortest_paths(authors_list: pd.Series, shortest_path=shortest_path, authors_mapping=authors_mapping):
    """17 s"""
    if isinstance(authors_list, list) and len(authors_list) == 2 and authors_list[0] and authors_list[1]:
        mapped_authors_1 = np.array([authors_mapping[author] for author in authors_list[0]])
        mapped_authors_2 = np.array([authors_mapping[author] for author in authors_list[1]])
        c = cartesian((mapped_authors_1,mapped_authors_2))
        return shortest_path[c[:,0],c[:,1]].tolist()[0]

In [324]:
st = time.time()
authors_list = train.source_authors_list.apply(lambda x: [x] if x is not None else None) + train.target_authors_list.apply(lambda x: [x] if x is not None else None)
train['colaboration_paths'] = authors_list.apply(extract_shortest_paths)
print(time.time() - st)

17.16381573677063
